In [ ]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import copy
from ultralytics import YOLO

In [ ]:
address = ""
model = YOLO('yolov5su.pt')  # Use a pretrained YOLOv5 model
def Tracking(img_address):
    all_files = glob.glob(img_address)
    img_row_size_org = 670
    img_col_size_org = 480
    img_row_size = 67
    img_col_size = 48
    img_size = img_row_size*img_col_size
    img_min_x = 15; img_max_x = 40
    img_min_y = 0; img_max_y = 50
    
    conf = 0.3
    
    img_real = []
    img_data = []
    lower_orange = np.array([10, 100, 100])
    upper_orange = np.array([25, 255, 255])
    for i in range(len(all_files)):
        img_black = cv.imread(all_files[i], cv.IMREAD_GRAYSCALE)
        img_color = cv.imread(all_files[i], cv.IMREAD_COLOR)
        img_rgb = cv.resize(cv.cvtColor(img_color, cv.COLOR_BGR2RGB), (img_col_size, img_row_size))[img_min_y:img_max_y,img_min_x:img_max_x,:]
        img_hsv = cv.resize(cv.cvtColor(img_color, cv.COLOR_BGR2HSV), (img_col_size, img_row_size))[img_min_y:img_max_y,img_min_x:img_max_x,:]
        mask = cv.inRange(img_hsv, lower_orange, upper_orange)
        output_image = np.zeros((img_hsv.shape[0], img_hsv.shape[1]), dtype=np.uint8)
        output_image[mask > 0] = 255
    
        img_real += [cv.resize(cv.cvtColor(img_color, cv.COLOR_BGR2RGB), (img_col_size_org, img_row_size_org))]
        img_data += [output_image]
    
    frames = {}; frames[0] = []
    similar_boxes = []
    fig, axes = plt.subplots(int(len(all_files)/(5.00001))+1, 5, figsize=(15, 5*int(len(all_files)/(5.00001))+1))
    axes = axes.flatten()
    for f in range(len(all_files)):
        # Perform object detection with lowered confidence threshold
        results = model(all_files[f], conf=conf)
        
        # Visualization
        ax1 = axes[f]
        ax1.imshow(img_real[f], extent=(0, img_col_size_org, img_row_size_org, 0), aspect='auto', alpha=0.3)
        ax1.set_aspect('auto')
        ax1.set_xlim(0, img_col_size_org)
        ax1.set_ylim(img_col_size_org, 0)
        ax2 = ax1.twinx().twiny()
        ax2.set_xlim(0, img_col_size)
        ax2.set_ylim(0, img_col_size)
        ax1.axes.xaxis.set_visible(False)
        ax1.axes.yaxis.set_visible(False)
        ax2.axes.xaxis.set_visible(False)
        ax2.axes.yaxis.set_visible(False)
        ax2.yaxis.set_ticks([])
        ax2.spines['right'].set_visible(False)
        plt.tight_layout(pad=0.1)

        # Bounding Boxes
        boxes_tensor = results[0].boxes.xyxy
        bounding_boxes = boxes_tensor.cpu().tolist()
        for box in bounding_boxes:
            min_x, min_y, max_x, max_y = box
            rect = plt.Rectangle(
                (min_x, min_y),
                max_x - min_x,
                max_y - min_y,
                fill=False, 
                edgecolor='red', 
                linewidth=2
            )
            plt.gca().add_patch(rect)
        plt.gca().invert_yaxis()
        frames[f+1] = bounding_boxes

        # Tracking
        current_boxes = frames[f+1]
        previous_boxes = frames[f]
    
        previous_similar_boxes = copy.deepcopy(similar_boxes)
        for i, current_box in enumerate(current_boxes):
            similar_boxes_last = [j[-1] for j in similar_boxes]
            current_new = 1
            for j, bbox in enumerate(similar_boxes_last):
                try:
                    if abs(bbox[0] - current_box[0]) <= 4 or abs(bbox[2] - current_box[2]) <= 4:
                        similar_boxes[j] += [current_box]
                        current_new = 0
                except:
                    current_new = 1
            if current_new==1:
                similar_boxes += [[current_box]]
        
        for i in range(len(previous_similar_boxes)):
            if previous_similar_boxes[i] == similar_boxes[i]:
                try:
                    prev_height = similar_boxes[i][-2][3] - similar_boxes[i][-2][1]
                    current_height = similar_boxes[i][-1][3] - similar_boxes[i][-1][1]
                    if current_height >= prev_height:
                        similar_boxes[i] = "IN"
                    elif current_height < prev_height:
                        similar_boxes[i] = "OUT"
                except:
                    if similar_boxes[i] != "IN" and similar_boxes[i] != "OUT":
                        similar_boxes[i] = "NONE"

        # Print results
        result_text = []
        for i in range(len(similar_boxes)):
            if similar_boxes[i] == 'IN' or similar_boxes[i] == 'OUT':
                result_text += [f"Object {i}: {similar_boxes[i]}\n"]
            elif similar_boxes[i] != 'NONE':
                result_text += [f"Object {i}: DETECT\n"]
        result_texts = "\n".join(result_text)
        plt.text(0.02, 0.95, result_texts, fontsize=18, fontweight='bold', color='red', ha='left', va='top')
        
        # for i, bbox in enumerate(bounding_boxes):
        #     if bbox[-2] > 0.1:
        #         # print(f"Cluster {i} centers: {cluster_centers[i]}")
        #         print(f"Cluster {i}: {bbox}")
    
        # for i in range(len(similar_boxes)):
        #     if similar_boxes[i] != 'NONE':
        #         print(f"Object {i}: {similar_boxes[i]}")

    for f in range(len(all_files), (int(len(all_files)/(5.00001))+1)*5):
        axes[f].axis('off')
        plt.tight_layout(pad=0.1)

    return fig

In [ ]:
%%time
all_folders = glob.glob(address+'all_class/*')
fig_tracking_in = Tracking(all_folders[0]+'/*.png')

In [ ]:
%%time
fig_tracking_in = Tracking(all_folders[1]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[2]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[3]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[4]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[5]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[6]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[7]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[8]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[9]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[10]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[11]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[12]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[13]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[14]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[15]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[16]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[17]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[18]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[19]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[20]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[21]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[22]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[23]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[24]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[25]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[26]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[27]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[28]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[29]+'/*.png')

In [ ]:
fig_tracking_in = Tracking(all_folders[30]+'/*.png')